In [7]:
!git clone https://huggingface.co/knkarthick/MEETING_SUMMARY

fatal: destination path 'MEETING_SUMMARY' already exists and is not an empty directory.


In [ ]:
!zip -r MEETING_SUMMARY.zip /content/MEETING_SUMMARY

  adding: content/MEETING_SUMMARY/ (stored 0%)
  adding: content/MEETING_SUMMARY/vocab.json (deflated 59%)
  adding: content/MEETING_SUMMARY/special_tokens_map.json (deflated 50%)
  adding: content/MEETING_SUMMARY/model.safetensors (deflated 7%)
  adding: content/MEETING_SUMMARY/pytorch_model.bin (deflated 7%)
  adding: content/MEETING_SUMMARY/tf_model.h5 (deflated 7%)
  adding: content/MEETING_SUMMARY/README.md (deflated 70%)
  adding: content/MEETING_SUMMARY/tokenizer_config.json (deflated 46%)
  adding: content/MEETING_SUMMARY/.git/ (stored 0%)
  adding: content/MEETING_SUMMARY/.git/lfs/ (stored 0%)
  adding: content/MEETING_SUMMARY/.git/lfs/tmp/ (stored 0%)
  adding: content/MEETING_SUMMARY/.git/lfs/incomplete/ (stored 0%)
  adding: content/MEETING_SUMMARY/.git/lfs/objects/ (stored 0%)
  adding: content/MEETING_SUMMARY/.git/lfs/objects/61/ (stored 0%)
  adding: content/MEETING_SUMMARY/.git/lfs/objects/61/58/ (stored 0%)
  adding: content/MEETING_SUMMARY/.git/lfs/objects/61/58/6158c

In [8]:
from transformers import pipeline
summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY")
text = '''The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.
'''
summarizer(text)


Device set to use cpu


[{'summary_text': 'The Eiffel Tower was built in 1889. It is the tallest structure in Paris and the second tallest free-standing structure in France after the Millau Viaduct. In 1930, it surpassed the Washington Monument to become the tallest man-made structure in the world. It was'}]

In [9]:
import gradio as gr
from transformers import pipeline
import torch # Or import tensorflow if using TF backend
import textwrap # For formatting description nicely

# --- Model Loading ---
# Load the summarization pipeline only once when the script starts
print("Loading1 the summarization model (knkarthick/MEETING_SUMMARY)...")
try:
    # Explicitly use GPU if available, otherwise CPU
    device = 0 if torch.cuda.is_available() else -1
    summarizer = pipeline(
        "summarization",
        model="knkarthick/MEETING_SUMMARY",
        device=device # Use device=0 for CUDA, device=-1 for CPU
    )
    print("Model loaded successfully.")
    model_loaded = True
except Exception as e:
    print(f"Error loading model: {e}")
    summarizer = None
    model_loaded = False
    # Provide a dummy summarizer for Gradio interface to load without crashing
    def dummy_summarizer(*args, **kwargs):
         return [{"summary_text": f"ERROR: Model knkarthick/MEETING_SUMMARY failed to load. Cannot summarize. Details: {e}"}]
    summarizer = dummy_summarizer


# --- Core Summarization Function ---
def summarize_meeting_text(text_to_summarize, min_len=30, max_len=150):
    """
    Takes text input and returns its summary using the loaded pipeline.
    Includes basic error handling and uses specified min/max length.
    """
    if not model_loaded and not callable(summarizer): # Check if it's the dummy
         # Error message is already in the dummy function's return
         summary_output = summarizer()
         return summary_output[0]['summary_text']

    if not text_to_summarize:
        return "Please enter some text (e.g., a meeting transcript) to summarize."

    print(f"Received text length: {len(text_to_summarize)}") # Optional: log input length
    print(f"Summarizing with min_length={min_len}, max_length={max_len}")

    try:
        # Ensure lengths are integers
        min_len = int(min_len)
        max_len = int(max_len)

        if min_len >= max_len:
            return "Error: Minimum summary length must be less than Maximum summary length."

        summary_output = summarizer(
            text_to_summarize,
            min_length=min_len,
            max_length=max_len,
            do_sample=False # Use greedy decoding for more deterministic output
        )
        # The pipeline returns a list with a dictionary
        if summary_output and isinstance(summary_output, list) and 'summary_text' in summary_output[0]:
            summary = summary_output[0]['summary_text']
            print(f"Generated summary length: {len(summary)}") # Optional: log output length
            return summary
        else:
             # Handle case where model loaded but output is unexpected
             if not model_loaded:
                 return summary_output[0]['summary_text'] # Return error from dummy
             else:
                return "Error: Received unexpected output from the summarizer."

    except Exception as e:
        print(f"Error during summarization: {e}")
        # Provide a more user-friendly error message
        if "maximum sequence length" in str(e):
             # Extract model's max length if possible (often in the error string)
             max_model_len = "the model's limit"
             try:
                 # Basic parsing attempt, might need adjustment based on exact error msg
                 parts = str(e).split(" ")
                 for i, part in enumerate(parts):
                     if part.isdigit() and "maximum sequence length" in " ".join(parts[i-5:i+1]):
                         max_model_len = f"{part} tokens"
                         break
             except Exception:
                 pass # Stick to default message if parsing fails
             return f"ERROR: Input text is too long ({len(text_to_summarize.split())} words approx). Please provide shorter text (max {max_model_len}). Details: {e}"
        return f"ERROR: An error occurred during summarization. Details: {e}"

# --- Gradio Interface Definition ---

# Use Blocks for more layout control, even if simple for now
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # Meeting Minutes Summarizer
        Paste your meeting transcript or notes below. The AI will generate a concise summary.
        *(Note: This tool creates a summary paragraph, not a fully structured document with separate fields like the example image.)*
        """
    )

    with gr.Row():
        with gr.Column(scale=2):
            input_text = gr.Textbox(
                lines=20,
                placeholder="Paste your raw meeting transcript or notes here...",
                label="Input Meeting Text"
            )
            with gr.Row():
                 min_length_slider = gr.Slider(minimum=10, maximum=100, value=30, step=5, label="Minimum Summary Length (tokens)")
                 max_length_slider = gr.Slider(minimum=50, maximum=500, value=150, step=10, label="Maximum Summary Length (tokens)")

            submit_button = gr.Button("Generate Summary", variant="primary")

        with gr.Column(scale=1):
            output_summary = gr.Textbox(
                label="Generated Summary",
                lines=25,
                interactive=False # Output box should not be editable by user
            )

    gr.Markdown("---") # Separator
    gr.Examples(
        examples=[
            [
                '''Alice: Okay team, let's kick off the weekly sync. Bob, any updates on the Q3 report?
Bob: Yes, the draft is almost ready. I need final figures from Carol by EOD tomorrow.
Carol: I'm working on it, Bob. Should have them for you by 4 PM tomorrow latest.
Alice: Great. And the marketing campaign, Dave?
Dave: We launched the social media ads yesterday. Initial engagement looks promising. Website traffic is up 15%. Key decisions were to focus on platform X and target demographic Y. Next steps are to analyze the first week's data and adjust budget allocation.
Alice: Excellent news. Anything else?
Bob: Just a reminder about the budget review meeting next Tuesday. Also, we need to finalize the speaker list for the upcoming webinar.
Alice: Right, thanks Bob. Okay, let's wrap up. Good progress everyone.''',
                30,
                130
            ],
            [
                '''Project Phoenix Kick-off Meeting Notes:
Attendees: Sarah (PM), Mark (Dev Lead), Lisa (QA), Tom (UX)
Absent: None
Date: 2023-10-26
Agenda: Project goals, timeline overview, initial role assignments, Q&A.
Discussion: Sarah presented the project charter, emphasizing the goal to improve user onboarding completion rate by 15% in Q1. Mark discussed the proposed tech stack (React frontend, Python backend). Lisa raised concerns about testing resources for the tight deadline. Tom presented initial wireframes and gathered feedback.
Decisions: Tech stack approved. Lisa to provide detailed QA resource estimate by Friday. Mark to set up repository and basic CI/CD. Tom to refine wireframes based on feedback.
Action Items: Lisa - QA estimate (Due EOW). Mark - Repo setup (Due EOD). Tom - Wireframe V2 (Due next Wed). Sarah - Schedule follow-up for next week.
''',
                50,
                200
            ]
        ],
        inputs=[input_text, min_length_slider, max_length_slider], # Ensure inputs match the function signature order for examples
        outputs=output_summary,
        fn=summarize_meeting_text, # The function to call when example is clicked
        cache_examples=False # Re-run examples if needed, or True if model/fn is deterministic
    )


    # Connect the button click to the function
    submit_button.click(
        fn=summarize_meeting_text,
        inputs=[input_text, min_length_slider, max_length_slider], # Pass current values from the UI
        outputs=output_summary
    )

# --- Launch the App ---
if __name__ == "__main__":
    print("Launching Gradio interface...")
    # share=True creates a public link (use with caution)
    demo.launch()
    # demo.launch(share=True) # Uncomment to get a public link

Loading1 the summarization model (knkarthick/MEETING_SUMMARY)...


Device set to use cpu


Model loaded successfully.
Launching Gradio interface...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1040fb62c6c5532b22.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.8 MB/s eta 0:00:00
